# Распознавание объектов на изображении

<table align="left"><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

Этот ноутбук использует [Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) и Вы шаг за шагом пройдете через процесс использования предварительно обученной модели для обнаружения объектов на изображении.

**Важно:** Это руководство предназначено для того, чтобы помочь вам сделать первый шаг к использованию API обнаружения объектов для построения моделей. Если вам просто нужна готовая модель, которая выполняет эту работу, см. [TFHub object detection example](https://colab.sandbox.google.com/github/tensorflow/hub/blob/master/examples/colab/object_detection.ipynb).

# Установки

Важно: Если вы работаете на локальном компьютере, обязательно следуйте инструкциям [инструкции по установке](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2.md). Этот ноутбук включает в себя только то, что необходимо для работы в Colab.

### Инсталляция TensorFlow

In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

Обязательно установите `pycocotools`

In [ ]:
!pip install pycocotools

Загрузите модели `tensorflow/models` или перейдите по команде `cd` в родительский каталог *репозитория*.

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Скомпилируйте protobuf и установите пакет object_detection

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Необязательные команды для контроля

In [ ]:
!ls -All

In [ ]:
%cd data/

In [ ]:
!pwd
!ls


In [ ]:
%ls
%cd object_detection/

### Импортируем нужные библиотеки

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Импортируем object detection module.

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Опции:

In [ ]:
# настроим опции tf1 в `utils.ops`
# tf.compat - модуль совместимости, позволяет писать код, который
# работает, как в TensorFlow 1.x, так и в 2.x.
utils_ops.tf = tf.compat.v1

# настроим путь к файлу gfile
tf.gfile = tf.io.gfile

# Подготовка модели

## Переменные base_url, model_file и др.

Любую модель, экспортируемую с помощью `export_inference_graph.py "  можно загрузить здесь, просто изменив путь.

BПо умолчанию мы используем здесь модель "SSD with Mobile net". Список других моделей, которые можно запускать "из коробки" с различной скоростью и точностью, см. в зоопарке моделей обнаружения. Смотрите [Зоопарк моделей распознавания](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) для получения списка других моделей, которые могут быть запущены с различной скоростью и точностью.

**Функция-загрузчик модели**

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

## Карта загрузочных меток
Карты меток сопоставляют индексы с названиями категорий, так что, когда наша сеть свертки предсказывает число 5, мы знаем, что это соответствует, например, самолету. Здесь мы используем внутренние служебные функции, возвращающие словарь, сопоставляющий целые числа с соответствующими строковыми метками.

In [ ]:
# Список строк, которые используются для добавления правильной метки для
# каждого поля
PATH_TO_LABELS = '/content/models/research/object_detection/data/mscoco_label_map.pbtxt'
# models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

Для простоты мы проведем тестирование на 3х изображениях:

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!ls /content/drive/MyDrive/TestRecPics

In [ ]:
# Если вы хотите протестировать код с вашими изображениями,
# просто добавьте путь к изображениям в TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/MyDrive/TestRecPics')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

# Распознавание

Загружаем модель нейронной сети «SSD with Mobile net»:

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

Проверяем входную сигнатуру модели, она ожидает пакет из 3-цветных изображений типа uint8:

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

Эта модель возвращает несколько выходных данных:

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

Добавим функцию-оболочку для вызова модели и очистки выходных данных:

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # На входе должен быть тензор,
  # конвертируем image используя `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # модель ожидает пакет изображений,
  # поэтому добавляем ось(измерение)
  # c помощью метода with `tf.newaxis`.

  input_tensor = input_tensor[tf.newaxis,...]

  # запускаем нейросеть для вывода результатов
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # Все выходные данные являются пакетом тензоров пакетов.
  # Преобразуем в массивы numpy и возьмем  индекс [0],
  # чтобы удалить пакетное измерение.
  # Нас интересуют только первое измерение - num_detections.

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes должен быть целочисленным
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Обработаем модель с помощью масок:
  if 'detection_masks' in output_dict:
    # Изменим формат маски bbox в соответствии
    # с размером изображения.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

Теперь напишем функцию, которая запускает предыдущую на каждом тестовом изображении и показывает результаты:

In [ ]:
def show_inference(model, image_path):
  # представление изображения на основе массива
  # будет использовано позже для подготовки
  # результирующего изображения с рамками и надписями на них.

  image_np = np.array(Image.open(image_path))
  # Запускаем распознавание
  output_dict = run_inference_for_single_image(model, image_np)
  # Визуализируем результаты распознавания
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


## Сегментация(выделение) распознанных экземпляров

Выбираем модель нейронной сети с сегментацией:

In [ ]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

Модель сегментации экземпляра включает в себя выходные данные detection_masks:

In [ ]:
masking_model.signatures['serving_default'].output_shapes

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)

In [ ]:
!ls -all

In [ ]:
!git clone https://github.com/Joeclinton1/google-images-download.git

In [ ]:
%cd google-images-download/

In [ ]:
!python setup.py install

In [ ]:
%cd google_images_download/

In [ ]:
!python google_images_download.py --keywords "jagermeister bottle" --limit 100 --format jpg

In [ ]:
!python google_images_download.py --keywords "pencil" --limit 6 --format jpg

Преобразование картинок

In [ ]:
from PIL import Image
import os
import argparse

In [ ]:
def rescale_images(directory, size):
    print(directory,' ',size)
    print(os.listdir(directory))
    i=1
    for img in os.listdir(directory):
        im = Image.open(directory+img)
        print(i,': '+directory+img,'\n')
        im_resized = im.resize(size, Image.ANTIALIAS)
        im_resized.save(directory+img)
        i+=1

In [ ]:
rescale_images('/content/google-images-download/google_images_download/downloads/pencil/', (800,600))

In [ ]:
%rm /content/google-images-download/google_images_download/downloads/pencil/*.*


In [ ]:
!pip install pyqt5


In [ ]:
!git clone https://github.com/tzutalin/labelImg.git

In [ ]:
%cp /content/google-images-download/google_images_download/downloads/pencil/* /content/drive/MyDrive/TestRecPics/pencil/